# Setup

In [4]:
import json
import tweepy
from tweepy import OAuthHandler
from pymongo import MongoClient
import os
from textblob import TextBlob
import re

In [5]:
# Twitter API setup
consumer_key = 'W1vEEpGJawjrqeW70yBUkfrWt'
consumer_secret = 'PSNivPYzJCjvEeQ5WW0zAhAIbAoEdzQBjTUG39up4yhnW9HABi'
access_token = '289683843-a9GiDa84Ak7FqD8c8Cu1qYzA5oEVqQ1FSKjkSudP'
access_secret = 'O3KhEusOxqTJGI6zvh8eqlDbyGyUT4tKThymbr57YFHWx'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [6]:
# MongoDB setup
os.system('mongo twitterdb --eval "db.dropDatabase();"')
client = MongoClient()
db = client.twitterdb

# Tweet Functions

In [7]:
def print_tweet(tweet):
    """
    Print text for a tweet
    """
    print(tweet['text'])


def store_tweet(tweet, collection):
    """
    Store tweet in MongoDB
    """
    db[collection].insert_one(tweet)


def limit_handled(cursor):
    """
    Code taken from tweepy to deal with rate limit issues
    """
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            print("Rate limit exceeded. Sleeping ...")
            time.sleep(15 * 60)


# Donald Trump Twitter

In [8]:
user = api.get_user('realDonaldTrump')

In [9]:
# Get trump tweets
n = 100  # number of tweets to get

for tweet in limit_handled(tweepy.Cursor(
    api.user_timeline, id='realDonaldTrump', count=n).items(n)):
    
    store_tweet(tweet=tweet._json, collection='trump_tweets')

In [10]:
# Get trump retweets
n = 100  # number of retweets to get

for tweet in db.trump_tweets.find():
    retweets = api.retweets(tweet['id_str'], count=n)
    
    for retweet in retweets:
        retweet = retweet._json
        retweet['trump_tweet_mongo_id'] = tweet['_id']
        
        store_tweet(tweet=retweet, 
                    collection="trump_retweets_" + str(tweet['_id']))

RateLimitError: [{u'message': u'Rate limit exceeded', u'code': 88}]

In [ ]:
# Export MongoDB
os.system('mongodump --db twitterdb --gzip --out "../data"')

## Sentiment Analysis

In [37]:
def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet['text']).split())


In [48]:
def get_tweet_sentiment(tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(clean_tweet(tweet))
        
        # just return sentiment score
        return analysis.sentiment.polarity
        

In [55]:
tweets = db.trump_tweets.find().limit(10)
#print tweets
for t in tweets:
    print get_tweet_sentiment(t)



0.0
0.533333333333
-0.01
-1.0
0.0125
-0.189583333333
0.48
0.85
0.85
0.0
